In [1]:
%load_ext autoreload
%autoreload 2
from rxn_modeling import train
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import json
import pickle as pkl
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score, confusion_matrix
from scipy.stats import gaussian_kde
import matplotlib.cm as cm
import time

In [ ]:
objectives = [
    "3_class",
    "4_class"
]
models = ["xgb", "lr", "rf", "nn"]
featurizations = ["mp_fraction", "pca"]

start = time.time()

results = []
for objective in objectives:
    for model in models:
        for featurization in featurizations:
            best_params, best_estimators, y_pred_train, y_pred_test, X_train_k, X_test_k, y_train_k, y_test_k = train(model=model, objective=objective, featurization=featurization)
            
            precision_scores, recall_scores, f1_scores = [], [], []
            for i, j in zip(y_test_k, y_pred_test):
                # change average to micro or macro
                res = precision_recall_fscore_support(i, j, average='micro')
                precision_scores.append(res[0])
                recall_scores.append(res[1])
                f1_scores.append(res[2])

            result = {
                "objective": objective,
                "model": model,
                "featurization": featurization,
                "precision": str(np.mean(precision_scores)),
                "precision_std": str(np.std(precision_scores)),
                "recall": str(np.mean(recall_scores)),
                "recall_std": str(np.std(recall_scores)),
                "f1_score": str(np.mean(f1_scores)),
                "f1_score_std": str(np.std(f1_scores)),
            }
            results.append(result)
            with open('data/rxn_condition_log_micro.json', 'w') as f:
                json.dump(results, f, indent=4)
            with open('data/time_log.txt', 'a') as f:
                f.writelines(str((time.time() - start)/60) + "\n")

Returning extracted data of 26787/31782 reactions.
After removing duplicates, remaining extracted data contains 16010/26787 reactions.
Returning extracted data of 6854/9518 reactions.
